In [1]:
#import the required libraries
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.combine import SMOTEENN

In [4]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()


In [5]:
df = spark.read.csv('tel_churn.csv',header=True,inferSchema=True)
df = df.toPandas()
df.head()

,_c0,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,0,29.85,29.85,0,1,0,0,1,1,...,0,0,1,0,1,0,0,0,0,0
1,1,0,56.95,1889.50,0,0,1,1,0,1,...,0,0,0,1,0,0,1,0,0,0
2,2,0,53.85,108.15,1,0,1,1,0,1,...,0,0,0,1,1,0,0,0,0,0
3,3,0,42.30,1840.75,0,0,1,1,0,1,...,1,0,0,0,0,0,0,1,0,0
4,4,0,70.70,151.65,1,1,0,1,0,1,...,0,0,1,0,1,0,0,0,0,0


In [6]:
df=df.drop('_c0',axis=1)

In [7]:
x=df.drop('Churn',axis=1)
x

,SeniorCitizen,MonthlyCharges,TotalCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29.85,29.85,1,0,0,1,1,0,1,...,0,0,1,0,1,0,0,0,0,0
1,0,56.95,1889.50,0,1,1,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
2,0,53.85,108.15,0,1,1,0,1,0,0,...,0,0,0,1,1,0,0,0,0,0
3,0,42.30,1840.75,0,1,1,0,1,0,1,...,1,0,0,0,0,0,0,1,0,0
4,0,70.70,151.65,1,0,1,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,0,84.80,1990.50,0,1,0,1,0,1,0,...,0,0,0,1,0,1,0,0,0,0
7028,0,103.20,7362.90,1,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,1
7029,0,29.60,346.45,1,0,0,1,0,1,1,...,0,0,1,0,1,0,0,0,0,0
7030,1,74.40,306.60,0,1,0,1,1,0,0,...,0,0,0,1,1,0,0,0,0,0


In [8]:
y=df['Churn']
y

0       0
1       0
2       1
3       0
4       1
       ..
7027    0
7028    0
7029    0
7030    1
7031    0
Name: Churn, Length: 7032, dtype: int32

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [11]:
#decisionTreeClassifier
model_dt=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8)

In [12]:
model_dt.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [13]:
y_pred=model_dt.predict(x_test)
y_pred

array([0, 0, 1, ..., 0, 0, 1])

In [14]:
model_dt.score(x_test,y_test)

0.7789623312011372

In [15]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.80      0.92      0.86      1010
           1       0.67      0.43      0.52       397

    accuracy                           0.78      1407
   macro avg       0.74      0.67      0.69      1407
weighted avg       0.77      0.78      0.76      1407



In [16]:
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_resample(x,y)

In [17]:
xr_train,xr_test,yr_train,yr_test=train_test_split(X_resampled, y_resampled,test_size=0.2)
print(xr_test,yr_test)

      SeniorCitizen  MonthlyCharges  TotalCharges  gender_Female  gender_Male  \
3499              0       81.300000    416.300000              0            1   
2770              0       50.800000     50.800000              0            1   
5119              0       70.150000     70.150000              1            0   
659               0       58.550000   3503.500000              1            0   
5234              1       25.231287     25.231287              1            0   
...             ...             ...           ...            ...          ...   
3706              0       80.581194    155.879647              1            0   
2592              0       49.200000   1498.550000              1            0   
2456              0       94.350000   5703.000000              1            0   
59                0       19.650000    244.800000              1            0   
1352              0       97.650000   6687.850000              0            1   

      Partner_No  Partner_Y

In [18]:
model_dt_smote=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8)

In [19]:
model_dt_smote.fit(xr_train,yr_train)
yr_predict = model_dt_smote.predict(xr_test)
model_score_r = model_dt_smote.score(xr_test, yr_test)
print(model_score_r)
print(metrics.classification_report(yr_test, yr_predict))

0.9402356902356902
              precision    recall  f1-score   support

           0       0.96      0.91      0.93       556
           1       0.93      0.96      0.94       632

    accuracy                           0.94      1188
   macro avg       0.94      0.94      0.94      1188
weighted avg       0.94      0.94      0.94      1188



In [20]:
print(metrics.confusion_matrix(yr_test, yr_predict))

[[508  48]
 [ 23 609]]


In [21]:
#RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

In [22]:
model_rf=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [23]:
model_rf.fit(x_train,y_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [24]:
y_pred=model_rf.predict(x_test)

In [25]:
model_rf.score(x_test,y_test)

0.7874911158493249

In [26]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.80      0.94      0.86      1010
           1       0.72      0.41      0.52       397

    accuracy                           0.79      1407
   macro avg       0.76      0.67      0.69      1407
weighted avg       0.78      0.79      0.77      1407



In [27]:
sm = SMOTEENN()
X_resampled1, y_resampled1 = sm.fit_resample(x,y)

In [28]:
xr_train1,xr_test1,yr_train1,yr_test1=train_test_split(X_resampled1, y_resampled1,test_size=0.2)

In [29]:
model_rf_smote=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [30]:
model_rf_smote.fit(xr_train1,yr_train1)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [31]:
yr_predict1 = model_rf_smote.predict(xr_test1)

In [32]:
model_score_r1 = model_rf_smote.score(xr_test1, yr_test1)

In [33]:
print(xr_test1, yr_test1)
print(model_score_r1)
print(metrics.classification_report(yr_test1, yr_predict1))

      SeniorCitizen  MonthlyCharges  TotalCharges  gender_Female  gender_Male  \
1406              0       19.400000    641.150000              1            0   
3908              0       48.592020    810.624369              1            0   
449               0       85.900000   6110.750000              1            0   
2066              0       75.800000   4415.750000              1            0   
5141              0       95.271545    897.133946              0            1   
...             ...             ...           ...            ...          ...   
165               0       65.200000   4543.150000              1            0   
201               0       24.900000   1174.800000              0            1   
1957              0       19.750000   1272.050000              1            0   
4485              0       81.235296    416.230675              0            1   
3998              0       74.448668   1133.478646              0            1   

      Partner_No  Partner_Y

In [34]:
print(metrics.confusion_matrix(yr_test1, yr_predict1))

[[488  42]
 [ 30 626]]


In [35]:
# Applying PCA
from sklearn.decomposition import PCA
pca = PCA(0.9)
xr_train_pca = pca.fit_transform(xr_train1)
xr_test_pca = pca.transform(xr_test1)
explained_variance = pca.explained_variance_ratio_

In [36]:
model=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [37]:
model.fit(xr_train_pca,yr_train1)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [38]:
yr_predict_pca = model.predict(xr_test_pca)

In [39]:
model_score_r_pca = model.score(xr_test_pca, yr_test1)

In [40]:
print(model_score_r_pca)
print(metrics.classification_report(yr_test1, yr_predict_pca))

0.7141652613827993
              precision    recall  f1-score   support

           0       0.73      0.56      0.64       530
           1       0.70      0.84      0.76       656

    accuracy                           0.71      1186
   macro avg       0.72      0.70      0.70      1186
weighted avg       0.72      0.71      0.71      1186



In [41]:
#pickling the model
import pickle

In [42]:
filename = 'model.sav'

In [43]:
pickle.dump(model_rf_smote, open(filename, 'wb'))

In [44]:
load_model = pickle.load(open(filename, 'rb'))

In [45]:
model_score_r1 = load_model.score(xr_test1, yr_test1)

In [46]:
model_score_r1

0.93929173693086

In [ ]:
##### Our final model i.e. RF Classifier with SMOTEENN, is now ready and dumped in model.sav.